In [1]:
# file kfkd.py
import os

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [2]:
# add to kfkd.py
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

1 #define _CUDA_NDARRAY_C
2 
3 #include <Python.h>
4 #include <structmember.h>
5 #include "theano_mod_helper.h"
6 
7 #include <numpy/arrayobject.h>
8 #include <iostream>
9 
10 #include "cuda_ndarray.cuh"
11 
12 #ifndef CNMEM_DLLEXPORT
13 #define CNMEM_DLLEXPORT
14 #endif
15 
16 #include "cnmem.h"
17 #include "cnmem.cpp"
18 
19 //If true, when there is a gpu malloc or free error, we print the size of allocated memory on the device.
20 #define COMPUTE_GPU_MEM_USED 0
21 
22 //If true, we fill with NAN allocated device memory.
23 #define ALLOC_MEMSET 0
24 
25 //If true, we print out when we free a device pointer, uninitialize a
26 //CudaNdarray, or allocate a device pointer
27 #define PRINT_FREE_MALLOC 0
28 
29 //If true, we do error checking at the start of functions, to make sure there
30 //is not a pre-existing error when the function is called.
31 //You probably need to set the environment variable
32 //CUDA_LAUNCH_BLOCKING=1, and/or modify the CNDA_THREAD_SYNC
33 //preprocessor macro 

808         Py_XINCREF(rval);
809         return rval;
810     }
811     else
812     {
813         PyObject * rval = CudaNdarray_Copy(self);
814         if (0) std::cerr << "DeepCopy created " << rval << " devdata " << ((CudaNdarray*)rval)->devdata << "\n";
815         if (NULL == rval)
816         {
817             Py_DECREF(selfkey);
818             return NULL;
819         }
820         if (PyDict_SetItem(memo, selfkey, rval))
821         {
822             Py_DECREF(rval);
823             Py_DECREF(selfkey);
824             return NULL;
825         }
826         Py_DECREF(selfkey);
827         return rval;
828     }
829 }
830 PyObject * CudaNdarray_ReduceSum(CudaNdarray * self, PyObject * py_reduce_mask)
831 {
832     if (!PySequence_Check(py_reduce_mask))
833     {
834         PyErr_SetString(PyExc_TypeError, "reduce_mask must be sequence of ints");
835         return NULL;
836     }
837     int len = PySequence_Length(py_reduce_mask);
838     if (len != self->nd)
839     {
840   

2008                             other_strides[2],
2009                             other_strides[3],
2010                             other_strides[4]);
2011                     CNDA_THREAD_SYNC;
2012                     cudaError_t err = cudaGetLastError();
2013                     if( cudaSuccess != err)
2014                     {
2015                         PyErr_Format(
2016                             PyExc_RuntimeError,
2017                             "CudaNdarray_inplace_elemwise case5: Cuda error: %s: %s. n_block=(%ld,%ld) n_threads=%ld\n",
2018                             "k5 with loop over k4",
2019                             cudaGetErrorString(err),
2020                             (long) n_blocks.x, (long) n_blocks.y, (long) n_threads.x);
2021                         Py_XDECREF(new_other);
2022                         return -1;
2023                     }
2024                 }
2025             }
2026             break;
2027         case 6:
2028             {
2029      

2992     PyObject* tuple = PyTuple_New(2);
2993     PyTuple_SetItem(tuple, 0, a);
2994     PyTuple_SetItem(tuple, 1, b);
2995     return tuple;
2996 }
2997 #endif
2998 
2999 static PyGetSetDef CudaNdarray_getset[] = {
3000     {"shape",
3001         (getter)CudaNdarray_get_shape,
3002         (setter)CudaNdarray_set_shape,
3003         "shape of this ndarray (tuple)",
3004         NULL},
3005     {"_strides",
3006         (getter)CudaNdarray_get_strides,
3007         (setter)CudaNdarray_set_strides,
3008         "data pointer strides (in elements)",
3009         NULL},
3010     {"strides",
3011         (getter)CudaNdarray_get_strides,
3012         (setter)CudaNdarray_set_strides,
3013         "data pointer strides (in elements)",
3014         NULL},
3015     //gpudata is needed to allow calling pycuda fct with CudaNdarray input.
3016     {"gpudata",
3017         (getter)CudaNdarray_get_dev_data,
3018         NULL,
3019         "device data pointer",
3020         NULL},
3021     {"_dev_

3772     if (err) {
3773         return err;
3774     }
3775 
3776     int typenum = PyArray_TYPE(obj);
3777     if (typenum != REAL_TYPENUM)
3778     {
3779         PyErr_SetString(PyExc_TypeError, "can only copy from float arrays");
3780         return -1;
3781     }
3782     assert( 4 ==  PyArray_ITEMSIZE(obj));
3783     PyArrayObject * py_src = (PyArrayObject *)PyArray_ContiguousFromAny(
3784         (PyObject*)obj, typenum, self->nd, self->nd);
3785     if (!py_src) {
3786         return -1;
3787     }
3788     npy_intp py_src_size = PyArray_SIZE(py_src);
3789     void *py_src_data = PyArray_DATA(py_src);
3790     cudaError_t cerr;
3791     CNDA_BEGIN_ALLOW_THREADS;
3792     cerr = cudaMemcpy(self->devdata, py_src_data,
3793                       py_src_size * sizeof(real),
3794                       cudaMemcpyHostToDevice);
3795     //CNDA_THREAD_SYNC;  // unneeded because cudaMemcpy is blocking anyway
3796     CNDA_END_ALLOW_THREADS;
3797     if (cudaSuccess != cerr)
3798     {


4682             case 1:
4683                 {
4684                     const int stride = a_str[reduce_dim0];
4685                     const float * a_data_i_max = a_data_i + dims_a[reduce_dim0] * stride;
4686                     while (a_data_i != a_data_i_max)
4687                     {
4688                         sum += a_data_i[0];
4689                         a_data_i += stride;
4690                     }
4691                 }
4692                 break;
4693             case 2:
4694                 {
4695                     int rd = reduce_dim0+1;
4696                     for (; rd < nd; ++rd)
4697                     {
4698                         if (a_str[rd] && (!z_str[rd])) // this means 'rd' is a dimension we are reducing over
4699                             break;
4700                     }
4701                     const int stride0 = a_str[reduce_dim0];
4702                     const int stride1 = a_str[rd];
4703                     for (int ii = 0; ii < dims_a[rd];

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc fatal   : Host compiler targets unsupported OS.

['nvcc', '-shared', '-O3', '-Xlinker', '/DEBUG', '-D HAVE_ROUND', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=mc72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,/Zi,/MD', '-I"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano\\lib\\site-packages\\theano\\sandbox\\cuda"', '-I"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano\\lib\\site-packages\\numpy\\core\\include"', '-I"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano\\include"', '-I"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano\\lib\\site-packages\\theano\\gof"', '-L"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano\\libs"', '-L"C:\\Users\\chuco\\Anaconda3\\envs\\bunny_theano"', '-o', 'C:\\Users\\chuco\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.15063-SP0-Intel64_Family_6_Model_

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'C:\\Users\\chuco\\AppData\\Local\\Temp\\try_flags_j154gotf.c:4:19: fatal error: cudnn.h: No such file or directory\r\ncompilation terminated.\r\n'
Mapped name None to device cuda: GeForce GTX 960 (0000:01:00.0)
C:\Users\chuco\Anaconda3\envs\bunny_theano\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from time import time
import numpy as np
import pickle
from nolearn.lasagne import BatchIterator
import theano
from sklearn.metrics import mean_squared_error

In [4]:
# Increase Python's recursion limit now that net6 is large enough
import sys
sys.setrecursionlimit(10000)

In [5]:
FTRAIN = 'C:/Users/chuco/Documents/MIDS-W207/Final_Project_FK/kaggle_data/training.csv'
FTEST = 'C:/Users/chuco/Documents/MIDS-W207/Final_Project_FK/kaggle_data/test.csv'

In [6]:
# Write function to load the data
# Normalize the target -- coordinates : current grid 96by96
# Normalize the features -- pixels : current range 255

def load(test = False, cols = None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN. 
    Pass a list of *cols* if you're only interested in a subset of the target columns"""
    
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname)) #Load pandas dataframe
    
    # The Image column has pixel values separated by space
    # convert these values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep = ' '))
    
    if cols: # get the useful columns out
        df = df[list(cols) + ['Image']]
        
    print(df.count()) # prints the number of values for each column
    df = df.dropna() 
    # drop all rows that have missing values in them. reduce from 7049 partial to 2140 full examples
    
    # make feature matrix X
    # first convert pixel values from pandas 1D series to numpy 2D array
    # normalize pixel valuaes
    X = np.vstack(df['Image'].values)/255. # scale pixel values to [0,1]
    X = X.astype(np.float32) # make sure numeric val are float32
    # X is numpy.ndarray, shape (2140, 9216) == 2140 examples, 96 grid * 96 grid
    
    
    if not test: # only train data has target columns
        
        y = df[df.columns[:-1]].values #last column is X, we want the y targets here
        # y is numpy.ndarray, shape(2140, 30) -- 7049 examples, 30 targets each
        
        y = (y - 48) / 48 # scale target coordinates to [-1.1] (currently 96 by 96)
        X, y = shuffle(X, y, random_state = 42) #shuffle train data
        y = y.astype(np.float32)
        
    else:
        y = None # test data has no targets available
        
    return X,y

In [7]:
# When using convolutional layers in Lasagne, we have to prepare the input data such 
# that each sample is no longer a flat vector of 9,216 pixel intensities, 
# but a three-dimensional matrix with shape (c, 0, 1), where c is the number of channels (colors), 
# and 0 and 1 correspond to the x and y dimensions of the input image. In our case, 
# the concrete shape will be (1, 96, 96), because we're dealing with a single (gray) color 
# channel only.

def load2d(test=False, cols=None):
    X, y = load(test=test)
    X = X.reshape(-1, 1, 96, 96)
    return X, y

In [8]:
flip_indices = [
    (0, 2), (1, 3),
    (4, 8), (5, 9), (6, 10), (7, 11),
    (12, 16), (13, 17), (14, 18), (15, 19),
    (22, 24), (23, 25),
    ]

# Lets see if we got it right
"""df = read_csv(os.path.expanduser(FTRAIN))
for i,j in flip_indices:
    print ("# {} -> {}".format(df.columns[i], df.columns[j]))"""

'df = read_csv(os.path.expanduser(FTRAIN))\nfor i,j in flip_indices:\n    print ("# {} -> {}".format(df.columns[i], df.columns[j]))'

In [9]:
class FlipBatchIterator(BatchIterator):
    flip_indices = [
        (0, 2), (1, 3),
        (4, 8), (5, 9), (6, 10), (7, 11),
        (12, 16), (13, 17), (14, 18), (15, 19),
        (22, 24), (23, 25),
        ]

    def transform(self, Xb, yb):
        Xb, yb = super(FlipBatchIterator, self).transform(Xb, yb)

        # Flip half of the images in this batch at random:
        bs = Xb.shape[0] #number of examples
        indices = np.random.choice(bs, bs // 2, replace=False) #choose half of them randomly
        Xb[indices] = Xb[indices, :, :, ::-1] #flip these guys randomly

        if yb is not None:
            # Horizontal flip of all x coordinates:
            yb[indices, ::2] = yb[indices, ::2] * -1

            # Swap places, e.g. left_eye_center_x -> right_eye_center_x
            for a, b in self.flip_indices:
                yb[indices, a], yb[indices, b] = (
                    yb[indices, b], yb[indices, a])

        return Xb, yb

In [10]:
def float32(k):
    return np.cast['float32'](k)

In [11]:
class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [12]:
class EarlyStopping(object):
    def __init__(self, patience=100):
        self.patience = patience
        self.best_valid = np.inf
        self.best_valid_epoch = 0
        self.best_weights = None

    def __call__(self, nn, train_history):
        current_valid = train_history[-1]['valid_loss']
        current_epoch = train_history[-1]['epoch']
        if current_valid < self.best_valid:
            self.best_valid = current_valid
            self.best_valid_epoch = current_epoch
            self.best_weights = nn.get_all_params_values()
        elif self.best_valid_epoch + self.patience < current_epoch:
            print("Early stopping.")
            print("Best valid loss was {:.6f} at epoch {}.".format(
                self.best_valid, self.best_valid_epoch))
            nn.load_params_from(self.best_weights)
            raise StopIteration()

In [13]:
net6 = NeuralNet(
    
    # Specify Architecture
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !DROPOUT
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !DROPOUT
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('dropout3', layers.DropoutLayer),  # !DROPOUT
        ('hidden4', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),  # !DROPOUT
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    dropout1_p=0.1,  # !DROPOUT PROBABILIY
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    dropout2_p=0.2,  # !
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    dropout3_p=0.3,  # !
    hidden4_num_units=500,
    dropout4_p=0.5,  # !
    hidden5_num_units=500,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=theano.shared(float32(0.03)), #Set startup update rate
    update_momentum=theano.shared(float32(0.9)), #Set startup momentum
    
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001), #Decrease update rate with epoch
        AdjustVariable('update_momentum', start=0.9, stop=0.999), #Increaes momentum with epoch
        EarlyStopping(patience=200) # Early Stopping
        ],
    
    # Regression NOT classification problem
    regression=True,
    
    # Flip half of the images in each batch
    batch_iterator_train=FlipBatchIterator(batch_size=128),
    
    
    max_epochs=3000,
    verbose=1,
    )

In [13]:
X, y = load2d()

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [15]:
time0 = time()
net6.fit(X, y)
print(time()-time0)

with open('net6.pickle', 'wb') as f:
    pickle.dump(net6, f, -1)

# Neural Network with 8051502 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x96x96
  1  conv1     32x94x94
  2  pool1     32x47x47
  3  dropout1  32x47x47
  4  conv2     64x46x46
  5  pool2     64x23x23
  6  dropout2  64x23x23
  7  conv3     128x22x22
  8  pool3     128x11x11
  9  dropout3  128x11x11
 10  hidden4   500
 11  dropout4  500
 12  hidden5   500
 13  output    30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.07058     0.03318    2.12686  7.20s
      2     0.01684     0.02074    0.81183  6.23s
      3     0.01048     0.01410    0.74323  6.23s
      4     0.00874     0.01043    0.83787  6.23s
      5     0.00783     0.00948    0.82547  6.23s
      6     0.00731     0.00774    0.94441  6.24s
      7     0.00701     0.00738    0.94992  6.23s
      8     0.00678     0.00693    0.97901  6.23s
      9     0.00655     0.00660    0.99343  6.24s
     10     0.0

    128     0.00434     0.00414    1.04978  6.34s
    129     0.00434     0.00414    1.04956  6.33s
    130     0.00434     0.00414    1.04628  6.36s
    131     0.00433     0.00414    1.04588  6.35s
    132     0.00432     0.00414    1.04450  6.29s
    133     0.00432     0.00413    1.04508  6.31s
    134     0.00433     0.00413    1.04827  6.36s
    135     0.00432     0.00413    1.04685  6.25s
    136     0.00431     0.00413    1.04544  6.40s
    137     0.00432     0.00412    1.04819  6.25s
    138     0.00432     0.00412    1.04880  6.24s
    139     0.00433     0.00413    1.04922  6.24s
    140     0.00431     0.00412    1.04496  6.23s
    141     0.00432     0.00412    1.04917  6.23s
    142     0.00431     0.00412    1.04581  6.23s
    143     0.00431     0.00412    1.04766  6.23s
    144     0.00431     0.00412    1.04740  6.23s
    145     0.00431     0.00412    1.04655  6.23s
    146     0.00432     0.00412    1.04846  6.23s
    147     0.00431     0.00412    1.04704  6.23s


    265     0.00400     0.00379    1.05574  6.36s
    266     0.00399     0.00378    1.05371  6.35s
    267     0.00397     0.00378    1.05032  6.35s
    268     0.00397     0.00378    1.05134  6.36s
    269     0.00397     0.00378    1.05164  6.37s
    270     0.00397     0.00377    1.05168  6.36s
    271     0.00397     0.00377    1.05168  6.36s
    272     0.00397     0.00377    1.05377  6.37s
    273     0.00396     0.00376    1.05282  6.35s
    274     0.00396     0.00376    1.05362  6.35s
    275     0.00394     0.00375    1.04847  6.36s
    276     0.00394     0.00375    1.05068  6.36s
    277     0.00395     0.00375    1.05431  6.35s
    278     0.00393     0.00374    1.04947  6.36s
    279     0.00394     0.00374    1.05462  6.36s
    280     0.00394     0.00373    1.05563  6.35s
    281     0.00393     0.00372    1.05410  6.36s
    282     0.00392     0.00372    1.05333  6.36s
    283     0.00393     0.00372    1.05801  6.35s
    284     0.00390     0.00371    1.05117  6.36s


    398     0.00329     0.00302    1.09054  6.23s
    399     0.00332     0.00302    1.09854  6.26s
    400     0.00329     0.00300    1.09692  6.24s
    401     0.00330     0.00299    1.10481  6.25s
    402     0.00328     0.00298    1.09980  6.24s
    403     0.00328     0.00298    1.10201  6.24s
    404     0.00328     0.00299    1.09664  6.23s
    405     0.00327     0.00298    1.09933  6.25s
    406     0.00327     0.00297    1.10119  6.26s
    407     0.00327     0.00295    1.10720  6.27s
    408     0.00325     0.00295    1.10335  6.23s
    409     0.00328     0.00294    1.11351  6.27s
    410     0.00324     0.00294    1.10010  6.23s
    411     0.00323     0.00294    1.09701  6.23s
    412     0.00324     0.00293    1.10452  6.23s
    413     0.00323     0.00292    1.10675  6.24s
    414     0.00322     0.00291    1.10585  6.23s
    415     0.00322     0.00291    1.10572  6.24s
    416     0.00324     0.00290    1.11504  6.23s
    417     0.00319     0.00291    1.09787  6.23s


    534     0.00272     0.00235    1.15801  6.22s
    535     0.00270     0.00235    1.14902  6.23s
    536     0.00268     0.00233    1.14712  6.23s
    537     0.00268     0.00234    1.14680  6.22s
    538     0.00269     0.00234    1.14709  6.22s
    539     0.00269     0.00233    1.15314  6.24s
    540     0.00270     0.00234    1.15479  6.23s
    541     0.00268     0.00233    1.15225  6.22s
    542     0.00267     0.00232    1.15042  6.23s
    543     0.00268     0.00232    1.15623  6.23s
    544     0.00266     0.00231    1.15251  6.23s
    545     0.00265     0.00231    1.14338  6.23s
    546     0.00265     0.00231    1.14685  6.23s
    547     0.00264     0.00231    1.14232  6.22s
    548     0.00267     0.00231    1.15613  6.22s
    549     0.00267     0.00231    1.15762  6.22s
    550     0.00264     0.00230    1.15083  6.23s
    551     0.00266     0.00230    1.15469  6.25s
    552     0.00266     0.00229    1.15778  6.22s
    553     0.00264     0.00229    1.15750  6.23s


...

   2233     0.00142     0.00123    1.15450  6.33s
   2234     0.00142     0.00123    1.15423  6.28s
   2235     0.00143     0.00123    1.15875  6.31s
   2236     0.00142     0.00123    1.15035  6.30s
   2237     0.00143     0.00123    1.15788  6.25s
   2238     0.00145     0.00123    1.17471  6.29s
   2239     0.00144     0.00123    1.16885  6.29s
   2240     0.00144     0.00123    1.16535  6.29s
   2241     0.00141     0.00123    1.14654  6.29s
   2242     0.00144     0.00123    1.17173  6.42s
   2243     0.00144     0.00123    1.17193  6.38s
   2244     0.00142     0.00123    1.15583  6.28s
   2245     0.00144     0.00123    1.16932  6.24s
   2246     0.00143     0.00123    1.16367  6.32s
   2247     0.00142     0.00123    1.15193  6.32s
   2248     0.00143     0.00123    1.15799  6.30s
   2249     0.00143     0.00123    1.15865  6.29s
   2250     0.00144     0.00124    1.16600  6.34s
   2251     0.00144     0.00123    1.16868  6.31s
   2252     0.00144     0.00123    1.17342  6.38s


   2391     0.00139     0.00121    1.14556  6.24s
   2392     0.00139     0.00121    1.15140  6.23s
   2393     0.00140     0.00121    1.15815  6.23s
   2394     0.00141     0.00121    1.16937  6.23s
   2395     0.00140     0.00121    1.15967  6.24s
   2396     0.00140     0.00121    1.16423  6.23s
   2397     0.00141     0.00121    1.16804  6.26s
   2398     0.00138     0.00121    1.14034  6.38s
   2399     0.00140     0.00121    1.15551  6.34s
   2400     0.00139     0.00121    1.15017  6.34s
   2401     0.00142     0.00121    1.17648  6.32s
   2402     0.00139     0.00121    1.15014  6.35s
   2403     0.00141     0.00120    1.17278  6.33s
   2404     0.00140     0.00120    1.16735  6.34s
   2405     0.00138     0.00121    1.14370  6.39s
   2406     0.00137     0.00121    1.13146  6.33s
   2407     0.00139     0.00121    1.14588  6.34s
   2408     0.00141     0.00121    1.16826  6.34s
   2409     0.00140     0.00121    1.16267  6.37s
   2410     0.00140     0.00121    1.16301  6.35s


   2549     0.00137     0.00119    1.15585  6.34s
   2550     0.00136     0.00119    1.15001  6.33s
   2551     0.00136     0.00118    1.14532  6.36s
   2552     0.00137     0.00118    1.15994  6.51s
   2553     0.00135     0.00118    1.14664  6.63s
   2554     0.00137     0.00118    1.15896  6.46s
   2555     0.00136     0.00118    1.15581  6.52s
   2556     0.00137     0.00118    1.15443  6.46s
   2557     0.00137     0.00118    1.16018  6.52s
   2558     0.00136     0.00118    1.14875  6.41s
   2559     0.00136     0.00118    1.15379  6.41s
   2560     0.00136     0.00118    1.15150  6.40s
   2561     0.00135     0.00118    1.14775  6.40s
   2562     0.00138     0.00118    1.16422  6.42s
   2563     0.00137     0.00118    1.15654  6.40s
   2564     0.00137     0.00118    1.16110  6.40s
   2565     0.00136     0.00118    1.15456  6.40s
   2566     0.00134     0.00118    1.13746  6.40s
   2567     0.00136     0.00118    1.15503  6.40s
   2568     0.00137     0.00118    1.15657  6.44s


   2707     0.00133     0.00116    1.14929  6.33s
   2708     0.00134     0.00116    1.15726  6.36s
   2709     0.00134     0.00116    1.15752  6.42s
   2710     0.00133     0.00116    1.14852  6.44s
   2711     0.00134     0.00116    1.15374  6.46s
   2712     0.00133     0.00116    1.14526  6.39s
   2713     0.00132     0.00116    1.13859  6.35s
   2714     0.00134     0.00116    1.15821  6.34s
   2715     0.00135     0.00116    1.15867  6.34s
   2716     0.00133     0.00116    1.14265  6.37s
   2717     0.00133     0.00116    1.14464  6.33s
   2718     0.00133     0.00116    1.13920  6.36s
   2719     0.00134     0.00116    1.15506  6.45s
   2720     0.00135     0.00116    1.15996  6.42s
   2721     0.00135     0.00116    1.16138  6.46s
   2722     0.00135     0.00116    1.16248  6.48s
   2723     0.00133     0.00116    1.14877  6.40s
   2724     0.00134     0.00116    1.16152  6.40s
   2725     0.00134     0.00116    1.15656  6.40s
   2726     0.00135     0.00116    1.17208  6.42s


   2865     0.00131     0.00114    1.14425  6.33s
   2866     0.00130     0.00114    1.13214  6.35s
   2867     0.00130     0.00115    1.13325  6.34s
   2868     0.00131     0.00115    1.14173  6.33s
   2869     0.00129     0.00115    1.12329  6.34s
   2870     0.00130     0.00115    1.13882  6.33s
   2871     0.00132     0.00114    1.15745  6.35s
   2872     0.00132     0.00114    1.16164  6.33s
   2873     0.00130     0.00114    1.14561  6.33s
   2874     0.00133     0.00114    1.17247  6.38s
   2875     0.00131     0.00114    1.15595  6.33s
   2876     0.00131     0.00114    1.15216  6.35s
   2877     0.00131     0.00114    1.15088  6.36s
   2878     0.00129     0.00114    1.13560  6.35s
   2879     0.00131     0.00114    1.15283  6.34s
   2880     0.00131     0.00114    1.14476  6.34s
   2881     0.00131     0.00114    1.14883  6.34s
   2882     0.00133     0.00114    1.16185  6.33s
   2883     0.00132     0.00114    1.15168  6.33s
   2884     0.00130     0.00114    1.14389  6.34s


In [15]:
net7 = NeuralNet(
    
    # Specify Architecture
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !DROPOUT
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !DROPOUT
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('dropout3', layers.DropoutLayer),  # !DROPOUT
        ('hidden4', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),  # !DROPOUT
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, 96, 96),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    dropout1_p=0.1,  # !DROPOUT PROBABILIY
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    dropout2_p=0.2,  # !
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    dropout3_p=0.3,  # !
    hidden4_num_units=1000,
    dropout4_p=0.5,  # !
    hidden5_num_units=1000,
    output_num_units=30, output_nonlinearity=None,

    update_learning_rate=theano.shared(float32(0.03)), #Set startup update rate
    update_momentum=theano.shared(float32(0.9)), #Set startup momentum
    
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001), #Decrease update rate with epoch
        AdjustVariable('update_momentum', start=0.9, stop=0.999), #Increaes momentum with epoch
        EarlyStopping(patience=200) # Early Stopping
        ],
    
    # Regression NOT classification problem
    regression=True,
    
    # Flip half of the images in each batch
    batch_iterator_train=FlipBatchIterator(batch_size=128),
    
    
    max_epochs=3000,
    verbose=1,
    )

In [16]:
time0 = time()
net7.fit(X, y)
print(time()-time0)

with open('net7.pickle', 'wb') as f:
    pickle.dump(net7, f, -1)

# Neural Network with 16561502 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x96x96
  1  conv1     32x94x94
  2  pool1     32x47x47
  3  dropout1  32x47x47
  4  conv2     64x46x46
  5  pool2     64x23x23
  6  dropout2  64x23x23
  7  conv3     128x22x22
  8  pool3     128x11x11
  9  dropout3  128x11x11
 10  hidden4   1000
 11  dropout4  1000
 12  hidden5   1000
 13  output    30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.06118     0.03790    1.61444  9.37s
      2     0.01403     0.02184    0.64254  8.15s
      3     0.00927     0.01478    0.62753  8.14s
      4     0.00778     0.01086    0.71667  8.15s
      5     0.00740     0.00996    0.74224  8.16s
      6     0.00712     0.00892    0.79774  8.14s
      7     0.00679     0.00946    0.71765  8.14s
      8     0.00668     0.00913    0.73151  8.15s
      9     0.00644     0.00944    0.68196  8.15s
     10    

    130     0.00427     0.00451    0.94763  7.18s
    131     0.00425     0.00452    0.94051  7.20s
    132     0.00425     0.00452    0.94054  7.51s
    133     0.00424     0.00451    0.93825  7.36s
    134     0.00426     0.00448    0.94975  7.31s
    135     0.00423     0.00447    0.94749  7.22s
    136     0.00425     0.00444    0.95777  7.22s
    137     0.00423     0.00447    0.94737  7.24s
    138     0.00423     0.00441    0.95962  7.29s
    139     0.00422     0.00439    0.96307  7.29s
    140     0.00421     0.00440    0.95847  7.28s
    141     0.00423     0.00439    0.96346  7.27s
    142     0.00421     0.00438    0.96120  7.29s
    143     0.00422     0.00436    0.96773  7.30s
    144     0.00421     0.00435    0.96745  7.20s
    145     0.00421     0.00436    0.96576  7.22s
    146     0.00421     0.00437    0.96203  7.22s
    147     0.00418     0.00433    0.96352  7.33s
    148     0.00421     0.00431    0.97606  7.25s
    149     0.00420     0.00430    0.97674  7.23s


    270     0.00365     0.00359    1.01676  7.32s
    271     0.00364     0.00358    1.01652  7.29s
    272     0.00364     0.00356    1.02207  7.31s
    273     0.00365     0.00356    1.02305  7.30s
    274     0.00363     0.00357    1.01800  7.30s
    275     0.00364     0.00355    1.02423  7.29s
    276     0.00362     0.00355    1.02098  7.31s
    277     0.00362     0.00354    1.02140  7.31s
    278     0.00361     0.00350    1.03190  7.31s
    279     0.00361     0.00352    1.02684  7.29s
    280     0.00361     0.00352    1.02757  7.30s
    281     0.00358     0.00350    1.02319  7.29s
    282     0.00358     0.00352    1.01765  7.31s
    283     0.00358     0.00349    1.02564  7.31s
    284     0.00358     0.00348    1.03003  7.30s
    285     0.00357     0.00351    1.01965  7.33s
    286     0.00356     0.00346    1.02688  7.34s
    287     0.00357     0.00348    1.02580  7.30s
    288     0.00355     0.00348    1.01875  7.32s
    289     0.00355     0.00348    1.02136  7.31s


    408     0.00279     0.00263    1.05932  7.28s
    409     0.00282     0.00262    1.07480  7.26s
    410     0.00281     0.00262    1.07220  7.39s
    411     0.00279     0.00262    1.06460  7.48s
    412     0.00279     0.00262    1.06810  7.51s
    413     0.00276     0.00261    1.05856  7.39s
    414     0.00276     0.00259    1.06887  7.42s
    415     0.00279     0.00258    1.07920  7.41s
    416     0.00277     0.00259    1.06960  7.27s
    417     0.00277     0.00257    1.07716  7.24s
    418     0.00275     0.00257    1.06904  7.28s
    419     0.00279     0.00258    1.07812  7.39s
    420     0.00276     0.00257    1.07312  7.34s
    421     0.00273     0.00255    1.07172  7.36s
    422     0.00274     0.00258    1.06241  7.32s
    423     0.00274     0.00257    1.06564  7.34s
    424     0.00273     0.00254    1.07722  7.32s
    425     0.00273     0.00256    1.06692  7.30s
    426     0.00274     0.00252    1.08417  7.30s
    427     0.00275     0.00255    1.07637  7.42s


    556     0.00234     0.00215    1.09141  7.43s
    557     0.00234     0.00216    1.08112  7.28s
    558     0.00235     0.00216    1.08866  7.27s
    559     0.00234     0.00215    1.08930  7.37s
    560     0.00234     0.00215    1.08712  7.39s
    561     0.00235     0.00214    1.09808  7.33s
    562     0.00233     0.00217    1.07302  7.28s
    563     0.00234     0.00215    1.08999  7.25s
    564     0.00234     0.00214    1.09083  7.33s
    565     0.00233     0.00214    1.08716  7.28s
    566     0.00232     0.00214    1.08219  7.31s
    567     0.00231     0.00213    1.08310  7.44s
    568     0.00233     0.00214    1.09141  6.57s
    569     0.00230     0.00214    1.07681  6.62s
    570     0.00233     0.00214    1.08770  6.56s
    571     0.00233     0.00215    1.08674  6.54s
    572     0.00231     0.00212    1.08616  6.47s
    573     0.00232     0.00213    1.08964  6.44s
    574     0.00234     0.00213    1.09776  6.58s
    575     0.00232     0.00211    1.09743  6.58s


...

   2282     0.00129     0.00117    1.09862  6.30s
   2283     0.00129     0.00118    1.09872  6.36s
   2284     0.00129     0.00117    1.10450  6.32s
   2285     0.00129     0.00117    1.10460  6.42s
   2286     0.00127     0.00117    1.09198  6.38s
   2287     0.00129     0.00117    1.09903  6.32s
   2288     0.00127     0.00118    1.08032  6.27s
   2289     0.00130     0.00117    1.11122  6.32s
   2290     0.00128     0.00116    1.09768  6.31s
   2291     0.00127     0.00117    1.08769  6.40s
   2292     0.00131     0.00117    1.11415  6.39s
   2293     0.00131     0.00117    1.12047  6.33s
   2294     0.00128     0.00117    1.09369  6.27s
   2295     0.00128     0.00117    1.09405  6.31s
   2296     0.00130     0.00118    1.10483  6.40s
   2297     0.00127     0.00117    1.08374  6.34s
   2298     0.00130     0.00117    1.11525  6.33s
   2299     0.00129     0.00117    1.10543  6.35s
   2300     0.00128     0.00117    1.09601  6.26s
   2301     0.00128     0.00117    1.09325  6.28s


   2442     0.00124     0.00115    1.07935  6.33s
   2443     0.00124     0.00115    1.07618  6.28s
   2444     0.00126     0.00115    1.09460  6.28s
   2445     0.00126     0.00115    1.10107  6.29s
   2446     0.00126     0.00115    1.10032  6.29s
   2447     0.00125     0.00115    1.08157  6.28s
   2448     0.00126     0.00115    1.09257  6.34s
   2449     0.00126     0.00115    1.09762  6.28s
   2450     0.00126     0.00115    1.10358  6.29s
   2451     0.00124     0.00114    1.08748  6.33s
   2452     0.00125     0.00115    1.09337  6.32s
   2453     0.00127     0.00115    1.10896  6.30s
   2454     0.00126     0.00115    1.09864  6.28s
   2455     0.00126     0.00114    1.10478  6.36s
   2456     0.00126     0.00114    1.10230  6.36s
   2457     0.00126     0.00114    1.09979  6.29s
   2458     0.00126     0.00115    1.09601  6.31s
   2459     0.00125     0.00115    1.08830  6.28s
   2460     0.00126     0.00115    1.09750  6.32s
   2461     0.00126     0.00114    1.09979  6.33s


   2602     0.00123     0.00112    1.09497  6.35s
   2603     0.00125     0.00112    1.11002  6.34s
   2604     0.00123     0.00113    1.09023  6.34s
   2605     0.00122     0.00113    1.07958  6.33s
   2606     0.00124     0.00113    1.10314  6.28s
   2607     0.00122     0.00112    1.08750  6.31s
   2608     0.00122     0.00112    1.08663  6.30s
   2609     0.00121     0.00112    1.07701  6.29s
   2610     0.00122     0.00113    1.08206  6.28s
   2611     0.00125     0.00113    1.11298  6.28s
   2612     0.00123     0.00113    1.08793  6.32s
   2613     0.00123     0.00113    1.09056  6.28s
   2614     0.00122     0.00112    1.08591  6.29s
   2615     0.00123     0.00112    1.09679  6.28s
   2616     0.00123     0.00112    1.09718  6.29s
   2617     0.00123     0.00112    1.09069  6.28s
   2618     0.00122     0.00112    1.08809  6.30s
   2619     0.00123     0.00112    1.09894  6.33s
   2620     0.00123     0.00112    1.10189  6.33s
   2621     0.00121     0.00112    1.08072  6.44s


   2762     0.00120     0.00110    1.08463  6.27s
   2763     0.00122     0.00111    1.09918  6.29s
   2764     0.00120     0.00111    1.08625  6.28s
   2765     0.00120     0.00111    1.08336  6.32s
   2766     0.00120     0.00110    1.08748  6.32s
   2767     0.00120     0.00110    1.08950  6.33s
   2768     0.00120     0.00110    1.09183  6.31s
   2769     0.00121     0.00110    1.10373  6.28s
   2770     0.00121     0.00110    1.09686  6.30s
   2771     0.00120     0.00110    1.08803  6.28s
   2772     0.00119     0.00110    1.08209  6.29s
   2773     0.00121     0.00110    1.09432  6.28s
   2774     0.00121     0.00110    1.09164  6.31s
   2775     0.00121     0.00110    1.09544  6.27s
   2776     0.00119     0.00110    1.08233  6.27s
   2777     0.00121     0.00110    1.09749  6.27s
   2778     0.00119     0.00110    1.07857  6.28s
   2779     0.00121     0.00110    1.10004  6.28s
   2780     0.00121     0.00110    1.09659  6.27s
   2781     0.00120     0.00110    1.08940  6.29s


   2922     0.00116     0.00109    1.07029  6.27s
   2923     0.00118     0.00109    1.08260  6.26s
   2924     0.00119     0.00109    1.09142  6.26s
   2925     0.00119     0.00109    1.09229  6.27s
   2926     0.00120     0.00109    1.10406  6.26s
   2927     0.00118     0.00109    1.08549  6.25s
   2928     0.00118     0.00109    1.08570  6.28s
   2929     0.00118     0.00109    1.08516  6.27s
   2930     0.00118     0.00108    1.08354  6.27s
   2931     0.00116     0.00108    1.07341  6.26s
   2932     0.00118     0.00108    1.09162  6.26s
   2933     0.00118     0.00108    1.09234  6.25s
   2934     0.00118     0.00108    1.09094  6.27s
   2935     0.00117     0.00108    1.07691  6.27s
   2936     0.00117     0.00108    1.08094  6.26s
   2937     0.00119     0.00108    1.09565  6.26s
   2938     0.00117     0.00108    1.08178  6.27s
   2939     0.00118     0.00109    1.08696  6.26s
   2940     0.00119     0.00109    1.09587  6.27s
   2941     0.00117     0.00109    1.07429  6.27s


In [17]:
import platform
platform.python_version()
#legg python 3.5.4

'3.5.4'